In [1]:
from itertools import product
from models.generator import SEMamba
import torch
from utils.util import load_config
import os
import librosa
from models.stfts import mag_phase_stft, mag_phase_istft
from models.pcs400 import cal_pcs

device = torch.device('cuda:1')
# config = '/disk4/chocho/SEMamba/exp/VCTK/dep3_h32_tf4_ds32_dc3_ex4/config.yaml'
# checkpoint_file = '/disk4/chocho/SEMamba/exp/VCTK/dep3_h32_tf4_ds32_dc3_ex4/g_00025000.pth'
config = '/disk4/chocho/SEMamba/exp/VCTK-400/dep4_h64_tf4_ds16_dc4_ex4/config.yaml'
checkpoint_file = "/disk4/chocho/SEMamba/exp/VCTK-400/dep4_h64_tf4_ds16_dc4_ex4/g_00093000.pth"
cfg = load_config(config)
n_fft, hop_size, win_size = cfg['stft_cfg']['n_fft'], cfg['stft_cfg']['hop_size'], cfg['stft_cfg']['win_size']
compress_factor = cfg['model_cfg']['compress_factor']
sampling_rate = cfg['stft_cfg']['sampling_rate']
model = SEMamba(cfg).to(device)
state_dict = torch.load(checkpoint_file, map_location=device)
model.load_state_dict(state_dict['generator'])
model.eval()
# output_folder = '/disk4/chocho/SEMamba/_202505/encoder-mamba-decoder'
# os.makedirs(output_folder, exist_ok=True)

SEMamba(
  (dense_encoder): DenseEncoder(
    (dense_conv_1): Sequential(
      (0): Conv2d(2, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): PReLU(num_parameters=64)
    )
    (dense_block): DenseBlock(
      (dense_block): ModuleList(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (2): PReLU(num_parameters=64)
        )
        (1): Sequential(
          (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 1), dilation=(2, 1))
          (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (2): PReLU(num_parameters=64)
        )
        (2): Sequential(
          (0): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(4, 1), dilation=(4, 1))
          (1)

In [2]:
feature_encoder = model.dense_encoder
mask_decoder = model.mask_decoder
phase_decoder = model.phase_decoder
TSMamba = model.TSMamba

In [ ]:
feature_encoder

DenseEncoder(
  (dense_conv_1): Sequential(
    (0): Conv2d(2, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (2): PReLU(num_parameters=64)
  )
  (dense_block): DenseBlock(
    (dense_block): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (2): PReLU(num_parameters=64)
      )
      (1): Sequential(
        (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 1), dilation=(2, 1))
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (2): PReLU(num_parameters=64)
      )
      (2): Sequential(
        (0): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(4, 1), dilation=(4, 1))
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

: 

In [3]:
                # x1 = x.clone()
                # b, c, t, f = x1.size()
                # x1 = x1.permute(0, 3, 2, 1).contiguous().view(b*f, t, c)
                # x1 = block.tlinear( block.time_mamba(x1).permute(0,2,1) ).permute(0,2,1) + x1
                # x1 = x1.view(b, f, t, c).permute(0, 2, 1, 3).contiguous().view(b*t, f, c)
                # x1 = block.flinear( block.freq_mamba(x1).permute(0,2,1) ).permute(0,2,1) + x1
                # x1 = x1.view(b, t, f, c).permute(0, 3, 1, 2)
                
                # x = block(x)
                # pps(x)
                # pp(id(x),id(x1))
                # pp((x != x1).sum())

In [9]:
from c66 import pp, pps
from einops import rearrange
with torch.no_grad():
    # for clean_or_noisy in  ["clean","noisy"]:
    for clean_or_noisy in  ["noisy"]:
        input_folder = f'/disk4/chocho/SEMamba/_test_feature_map_{clean_or_noisy}'
        for i, fname in enumerate(os.listdir( input_folder )):
            print(input_folder, fname)
            noisy_wav, _ = librosa.load(os.path.join( input_folder, fname ), sr=sampling_rate)
            pps(noisy_wav)
            noisy_wav = torch.FloatTensor(noisy_wav).to(device)
            
            norm_factor = torch.sqrt(len(noisy_wav) / torch.sum(noisy_wav ** 2.0)).to(device)
            pp(norm_factor)
            noisy_wav = (noisy_wav * norm_factor).unsqueeze(0)
            pps(noisy_wav)
            noisy_mag, noisy_pha, noisy_com = mag_phase_stft(noisy_wav, n_fft, hop_size, win_size, compress_factor)
            
            # feature_encoder
            # Reshape inputs
            noisy_mag = rearrange(noisy_mag, 'b f t -> b t f').unsqueeze(1)  # [B F T] -> [B, 1, T, F]
            noisy_pha = rearrange(noisy_pha, 'b f t -> b t f').unsqueeze(1)  # [B F T] -> [B, 1, T, F]
            pps(noisy_mag, noisy_pha)

            # Concatenate magnitude and phase inputs
            x = torch.cat((noisy_mag, noisy_pha), dim=1)  # [B, 2, T, F]
            pps(x)

            # Feature Encoder
            x = feature_encoder(x)
            pps(x)
            
            # TF-Mamba
            # TSMamba is a instance of TFMambaBlock
            for block in TSMamba:
                b, c, t, f = x.size()
                x = x.permute(0, 3, 2, 1).contiguous().view(b*f, t, c)
                print("---")
                pps(x)
                x = block.tlinear( block.time_mamba(x).permute(0,2,1) ).permute(0,2,1) + x
                pps(x)
                x = x.view(b, f, t, c).permute(0, 2, 1, 3).contiguous().view(b*t, f, c)
                x = block.flinear( block.freq_mamba(x).permute(0,2,1) ).permute(0,2,1) + x
                pps(x)
                x = x.view(b, t, f, c).permute(0, 3, 1, 2)
                pps(x)
                print("---")

            # Mag, Pha Decoder
            denoised_mag = rearrange(mask_decoder(x) * noisy_mag, 'b c t f -> b f t c').squeeze(-1)
            denoised_pha = rearrange(phase_decoder(x), 'b c t f -> b f t c').squeeze(-1)
            pps(noisy_mag)
            pps(mask_decoder(x), phase_decoder(x))
            pps(denoised_mag, denoised_pha)
            
            # 
            audio_g = mag_phase_istft(denoised_mag, denoised_pha, n_fft, hop_size, win_size, compress_factor)
            pps(audio_g)
            audio_g = audio_g / norm_factor
            audio_g = cal_pcs(audio_g.squeeze().cpu().numpy())
            pps(audio_g)
            
            break
            
        break

/disk4/chocho/SEMamba/_test_feature_map_noisy p226_018.wav
noisy_wav's shape: (94035,)
norm_factor: 20.062986373901367
noisy_wav's shape: torch.Size([1, 94035])
noisy_mag's shape: torch.Size([1, 1, 941, 257])
noisy_pha's shape: torch.Size([1, 1, 941, 257])
x's shape: torch.Size([1, 2, 941, 257])
x's shape: torch.Size([1, 32, 941, 128])
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
noisy_mag'

In [ ]:
from c66 import pp, pps
from einops import rearrange
with torch.no_grad():
    # for clean_or_noisy in  ["clean","noisy"]:
    for clean_or_noisy in  ["noisy"]:
        input_folder = f'/disk4/chocho/SEMamba/_test_feature_map_{clean_or_noisy}'
        for i, fname in enumerate(os.listdir( input_folder )):
            print(input_folder, fname)
            noisy_wav, _ = librosa.load(os.path.join( input_folder, fname ), sr=sampling_rate)
            noisy_wav = torch.FloatTensor(noisy_wav).to(device)
            
            norm_factor = torch.sqrt(len(noisy_wav) / torch.sum(noisy_wav ** 2.0)).to(device)
            # noisy_wav: [len_wav,] = [L,]
            # norm_factor: [1,]
            # !!!
            
            noisy_wav = (noisy_wav * norm_factor).unsqueeze(0)
            # noisy_wav: [1, len_wav] = [1, L]
            # !!!

            noisy_mag, noisy_pha, noisy_com = mag_phase_stft(noisy_wav, n_fft, hop_size, win_size, compress_factor)
            # noisy_mag, noisy_pha: [1, F, T] = [1, n_fft//2, len_wav//hop_size]
            # !!!
            
            # feature_encoder
            # Reshape inputs
            noisy_mag = rearrange(noisy_mag, 'b f t -> b t f').unsqueeze(1)  # [B F T] -> [B, 1, T, F]
            noisy_pha = rearrange(noisy_pha, 'b f t -> b t f').unsqueeze(1)  # [B F T] -> [B, 1, T, F]

            # Concatenate magnitude and phase inputs
            x = torch.cat((noisy_mag, noisy_pha), dim=1)  # [B, 2, T, F]

            # Feature Encoder
            x = feature_encoder(x)
            # [B, 2, T, F] -> [B, h, T, F//2]
            # !!!
            
            # TF-Mamba
            # TSMamba is a instance of TFMambaBlock
            for block in TSMamba:
                b, c, t, f = x.size()
                # b, c, t, f = [1, h, T, F]
                
                x = x.permute(0, 3, 2, 1).contiguous().view(b*f, t, c)
                
                x = block.tlinear( block.time_mamba(x).permute(0,2,1) ).permute(0,2,1) + x
                # [F, T, h] -> [F, T, h]
                # !!!
                
                x = x.view(b, f, t, c).permute(0, 2, 1, 3).contiguous().view(b*t, f, c)
                
                x = block.flinear( block.freq_mamba(x).permute(0,2,1) ).permute(0,2,1) + x
                # [T, F, h] -> [T, F, h]
                # !!!
                
                x = x.view(b, t, f, c).permute(0, 3, 1, 2)

            # Mag, Pha Decoder
            denoised_mag = rearrange(mask_decoder(x) * noisy_mag, 'b c t f -> b f t c').squeeze(-1)
            # [1, 1, T, F] * [1, 1, T, F] = [1, 1, T, F] -> [1, F, T, 1] -> [1, F, T]
            # !!!
            
            denoised_pha = rearrange(phase_decoder(x), 'b c t f -> b f t c').squeeze(-1)
            # [1, 1, T, F] -> [1, F, T, 1] -> [1, F, T]
            # !!!
            
            audio_g = mag_phase_istft(denoised_mag, denoised_pha, n_fft, hop_size, win_size, compress_factor)
            # [1, ~L]
            # !!!
            
            audio_g = audio_g / norm_factor
            # !!!
            
            audio_g = cal_pcs(audio_g.squeeze().cpu().numpy())
            # [~L,]
            # !!!
            
            pps(audio_g)
            
            break
            
        break

/disk4/chocho/SEMamba/_test_feature_map_noisy p226_018.wav
norm_factor: 20.062986373901367
noisy_wav's shape: torch.Size([1, 94035])
x's shape: torch.Size([1, 2, 941, 257])
x's shape: torch.Size([1, 32, 941, 128])
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([941, 128, 32])
x's shape: torch.Size([1, 32, 941, 128])
---
denoised_mag's shape: torch.Size([1, 257, 941])
denoised_pha's shape: torch.Size([1, 257, 941])
audio_g's shape: (94000,)


In [ ]:
from calflops import calculate_flops
with torch.no_grad():
    # 使用 calflops 計算 FLOPs，將 args 改為列表
        flops, macs, params = calculate_flops(
            model=model,
            args=[noisy_mag, noisy_pha],  # 使用列表而非元組
            print_results=True  # 顯示逐層結果
        )
        print(f"Total FLOPs for {fname}: {flops}")
        print(f"Total Params: {params}")
        print(f"Total MACs: {macs}")

EinopsError:  Error while processing rearrange-reduction pattern "b f t -> b t f".
 Input tensor shape: torch.Size([1, 1, 941, 257]). Additional info: {}.
 Wrong shape: expected 3 dims. Received 4-dim tensor.

In [ ]:
flops, macs, params = calculate_flops(
    model=model,
    args=[noisy_mag, noisy_pha],  # 使用列表而非元組
    print_results=True  # 顯示逐層結果
)
print(f"Total FLOPs for {fname}: {flops}")
print(f"Total Params: {params}")
print(f"Total MACs: {macs}")

In [6]:
from c66 import pp, pps
from einops import rearrange
from calflops import calculate_flops

with torch.no_grad():
    # for clean_or_noisy in  ["clean","noisy"]:
    for clean_or_noisy in  ["noisy"]:
        input_folder = f'/disk4/chocho/SEMamba/_test_feature_map_{clean_or_noisy}'
        for i, fname in enumerate(os.listdir( input_folder )):
            print(input_folder, fname)
            noisy_wav, _ = librosa.load(os.path.join( input_folder, fname ), sr=sampling_rate)
            pps(noisy_wav)
            noisy_wav = torch.FloatTensor(noisy_wav).to(device)
            
            norm_factor = torch.sqrt(len(noisy_wav) / torch.sum(noisy_wav ** 2.0)).to(device)
            pp(norm_factor)
            noisy_wav = (noisy_wav * norm_factor).unsqueeze(0)
            pps(noisy_wav)
            noisy_mag, noisy_pha, noisy_com = mag_phase_stft(noisy_wav, n_fft, hop_size, win_size, compress_factor)
            
            # feature_encoder
            # Reshape inputs
            noisy_mag = rearrange(noisy_mag, 'b f t -> b t f').unsqueeze(1)  # [B F T] -> [B, 1, T, F]
            noisy_pha = rearrange(noisy_pha, 'b f t -> b t f').unsqueeze(1)  # [B F T] -> [B, 1, T, F]

            # Concatenate magnitude and phase inputs
            x = torch.cat((noisy_mag, noisy_pha), dim=1)  # [B, 2, T, F]
            pps(x)

            # Feature Encoder
            x = feature_encoder(x)
            pps(x)
            
            # TF-Mamba
            # TSMamba is a instance of TFMambaBlock
            for block in TSMamba:
                b, c, t, f = x.size()
                x = x.permute(0, 3, 2, 1).contiguous().view(b*f, t, c)
                print("---")
                pps(x)
                x = block.tlinear( block.time_mamba(x).permute(0,2,1) ).permute(0,2,1) + x
                pps(x)
                
                flops, macs, params = calculate_flops(
                    model=block.time_mamba,
                    args=[x],  # 使用列表而非元組
                    print_results=True  # 顯示逐層結果
                )
                print(f"Total FLOPs for {fname}: {flops}")
                print(f"Total Params: {params}")
                print(f"Total MACs: {macs}")
                
                x = x.view(b, f, t, c).permute(0, 2, 1, 3).contiguous().view(b*t, f, c)
                x = block.flinear( block.freq_mamba(x).permute(0,2,1) ).permute(0,2,1) + x
                pps(x)
                x = x.view(b, t, f, c).permute(0, 3, 1, 2)
                pps(x)
                print("---")

            # Mag, Pha Decoder
            denoised_mag = rearrange(mask_decoder(x) * noisy_mag, 'b c t f -> b f t c').squeeze(-1)
            denoised_pha = rearrange(phase_decoder(x), 'b c t f -> b f t c').squeeze(-1)
            pps(denoised_mag, denoised_pha)
            
            # 
            audio_g = mag_phase_istft(denoised_mag, denoised_pha, n_fft, hop_size, win_size, compress_factor)
            audio_g = audio_g / norm_factor
            audio_g = cal_pcs(audio_g.squeeze().cpu().numpy())
            pps(audio_g)
            
            break
            
        break

/disk4/chocho/SEMamba/_test_feature_map_noisy p226_018.wav
noisy_wav's shape: (94035,)
norm_factor: 20.062986373901367
noisy_wav's shape: torch.Size([1, 94035])
x's shape: torch.Size([1, 2, 941, 257])
x's shape: torch.Size([1, 32, 941, 128])
---
x's shape: torch.Size([128, 941, 32])
x's shape: torch.Size([128, 941, 32])

------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  51.78 K 
fwd MACs:                                                               3.02 GMACs
fwd FLOPs:                                       